# Plots
<hr style="clear:both">

This notebook presents and displays our plot illustrating out story.

**Project Mentor:** [Shuo Wen](http://personnes.epfl.ch/shuo.wen) ([Email](shuo.wen@epfl.ch)),
**Authors:** Mahmoud Dokmak, Matthieu Borello, Léo Brunneau, Loïc Domingos, Bastien Armstrong

<hr style="clear:both">

In [3]:
import pandas as pd

movies = pd.read_pickle('./data/Success_Metric.pkl')
movies.head()

,freebase_movie_id,imdb_id,item_id,movie_name,combined_release_date,Box_Office,inflation_adjusted_box_office_revenue,movie_countries_final,movie_genres_final,sa_score,popularity,rating,success_score,success_metric_scaled
30973,/m/083skw,031381,NaN,Gone with the Wind,1939-12-15,0.143767,0.996126,United States of America,"[Film adaptation, Costume drama, Roadshow thea...",0.619947,1.0,0.785789,22.186218,5.000000
9888,/m/0k4d7,029583,NaN,Snow White and the Seven Dwarfs,1937-12-21,0.149518,1.000000,United States of America,"[Romance Film, Musical, Fantasy, Family Film, ...",0.619947,1.0,0.696421,21.949103,4.944015
1036,/m/04j1zjw,034492,NaN,Bambi,1942-08-14,0.096125,0.567962,United States of America,"[Adventure, Children's/Family, Animation, Comi...",0.619947,1.0,0.684211,18.456170,4.119294
66323,/m/0dtfn,076759,NaN,Star Wars Episode IV: A New Hope,1977-05-25,0.278692,0.442916,United States of America,"[Science Fiction, Adventure, Fantasy, Coming o...",0.619947,1.0,0.810947,17.836009,3.972866
18216,/m/0dr_4,120338,1721.0,Titanic,1997-11-18,0.785462,0.471324,United States of America,"[Tragedy, Costume drama, Historical fiction, A...",0.592075,1.0,0.714080,17.689057,3.938169
